<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1u10wrxIRiRcZTLO_0_85UafCP7yO99pp?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)

# Testing Claude 4.5 Haiku Model Using OpenRouter

This notebook provides a comprehensive guide to using the Claude 4.5 Haiku model via OpenRouter's API within the LangChain framework. We will cover everything from basic setup to advanced examples including agent workflows, tool calling, and search integration.

**Note:** You will need an API key from [OpenRouter](https://openrouter.ai/) to run the examples.

### Installation

First, let's install the necessary Python libraries for Claude 4.5 Haiku testing.

In [ ]:
%pip install langchain langchain-openai langchain-community tavily-python

### Basic Usage with ChatOpenAI and OpenRouter

Here's how to set up the `ChatOpenAI` class to connect to the Claude 4.5 Haiku model through OpenRouter."

In [ ]:
import os
from langchain_openai import ChatOpenAI
from google.colab import userdata

# It's recommended to set your API key as an environment variable
api_key = userdata.get("OPENROUTER_API_KEY")

# Initialize the ChatOpenAI model for Claude 4.5 Haiku
claude_llm = ChatOpenAI(
    model="anthropic/claude-haiku-4.5",
    openai_api_key=api_key,
    openai_api_base="https://openrouter.ai/api/v1"
)

# Let's test it with a simple prompt
response = claude_llm.invoke("What is the Model Context Protocol?")
print(response.content)

### Multilingual Capabilities

Claude 4.5 Haiku is proficient in multiple languages. Let's test this by sending prompts in Hindi and Spanish."

In [ ]:
# Example in Hindi
hindi_prompt = "कृत्रिम बुद्धिमत्ता क्या है?" # Translation: What is Artificial Intelligence?
hindi_response = claude_llm.invoke(hindi_prompt)
print(f"Response in Hindi:\n{hindi_response.content}")

print("\n" + "="*50 + "\n")

# Example in Spanish
spanish_prompt = "¿Cuál es la capital de Argentina?" # Translation: What is the capital of Argentina?
spanish_response = claude_llm.invoke(spanish_prompt)
print(f"Response in Spanish:\n{spanish_response.content}")

### Building a Simple Chatbot

We can create a simple conversational chatbot by managing the chat history."

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant that speaks like a pirate."),
    HumanMessage(content="Ahoy! What's your name?"),
]

# First turn
response = claude_llm.invoke(messages)
print(f"Pirate Bot: {response.content}")

# Add the bot's response to the history
messages.append(response)

# Second turn
messages.append(HumanMessage(content="What's the weather like today?"))
response = claude_llm.invoke(messages)
print(f"\nPirate Bot: {response.content}")

## Advanced Examples"

### Tool Calling and Tavily Search Using Claude 4.5 Haiku

Let's create an agent that can use tools, specifically the Tavily search tool for real-time information retrieval."

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import TavilySearchResults
import os

# 1. Setup LLM via OpenRouter
llm = ChatOpenAI(
    model="anthropic/claude-haiku-4.5",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=api_key,
)

# 2. Setup Tavily Search Tool
# Note: You'll need a Tavily API key - get one at https://tavily.com/
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
tavily_tool = TavilySearchResults(max_results=2)

# 3. Create Agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant with access to search tools."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
agent = create_tool_calling_agent(llm, [tavily_tool], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[tavily_tool], verbose=True)

# 4. Run Agent
response = agent_executor.invoke({"input": "What are the latest developments in AI safety research in 2024?"})
print(response['output'])

### Custom Tool Creation and Agent Workflow

Let's create a custom tool and demonstrate more advanced agent capabilities."

In [ ]:
from langchain.tools import tool
import math

@tool
def calculate_area(shape: str, **kwargs) -> str:
    """Calculate the area of different shapes.

    Args:
        shape: The shape to calculate area for (circle, rectangle, triangle)
        **kwargs: Shape-specific parameters
            - For circle: radius
            - For rectangle: length, width
            - For triangle: base, height
    """
    shape = shape.lower()

    if shape == "circle":
        radius = kwargs.get("radius", 0)
        area = math.pi * radius ** 2
        return f"The area of a circle with radius {radius} is {area:.2f} square units."

    elif shape == "rectangle":
        length = kwargs.get("length", 0)
        width = kwargs.get("width", 0)
        area = length * width
        return f"The area of a rectangle with length {length} and width {width} is {area} square units."

    elif shape == "triangle":
        base = kwargs.get("base", 0)
        height = kwargs.get("height", 0)
        area = 0.5 * base * height
        return f"The area of a triangle with base {base} and height {height} is {area} square units."

    else:
        return f"Sorry, I don't know how to calculate the area of a {shape}."

# Create agent with multiple tools
tools = [tavily_tool, calculate_area]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Test the agent with multiple tool usage
response = agent_executor.invoke({
    "input": "Calculate the area of a circle with radius 5, and then search for information about the mathematical constant pi."
})
print(response['output'])

### Advanced Parameter Tuning

You can control the model's output by tuning parameters like `temperature` and `max_tokens`.

- **`temperature`**: Controls randomness. Lower values (e.g., 0.1) make the output more deterministic, while higher values (e.g., 0.9) make it more creative.
- **`max_tokens`**: Sets the maximum length of the generated response."

In [ ]:
# Creative response with high temperature
creative_llm = ChatOpenAI(
    model="anthropic/claude-haiku-4.5",
    openai_api_key=api_key,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.9,
    max_tokens=150
)

prompt = "Write a short, futuristic story about a cat who can code."
creative_response = creative_llm.invoke(prompt)
print(f"Creative Story:\n{creative_response.content}")

print("\n" + "="*50 + "\n")

# Factual response with low temperature
factual_llm = ChatOpenAI(
    model="anthropic/claude-haiku-4.5",
    openai_api_key=api_key,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.1
)

prompt = "Explain the theory of relativity in simple terms."
factual_response = factual_llm.invoke(prompt)
print(f"Factual Explanation:\n{factual_response.content}")

### Code Generation

Let's test Claude 4.5 Haiku's code generation capabilities."

In [ ]:
prompt = "Write a Python function that takes a list of numbers and returns the sum of all even numbers in the list."
code_response = claude_llm.invoke(prompt)
print(f"Generated Code:\n{code_response.content}")

### Few-Shot Prompting

Few-shot prompting provides the model with examples to guide its response format."

In [12]:
prompt = """Translate the following English words to French:

"sea -> mer"
"sky -> ciel"
"book -> livre"
"house ->"
"""

few_shot_response = claude_llm.invoke(prompt)
print(f"Translation of 'house': {few_shot_response.content}")

Translation of 'house': "house -> maison"


### Streaming Responses

For longer responses, you can use streaming to get real-time output."

In [ ]:
import time

# Create a streaming version of the model
streaming_llm = ChatOpenAI(
    model="anthropic/claude-haiku-4.5",
    openai_api_key=api_key,
    openai_api_base="https://openrouter.ai/api/v1",
    streaming=True
)

prompt = "Write a detailed explanation of how machine learning works, covering the key concepts."

print("Streaming response:")
print("-" * 50)

# Stream the response
for chunk in streaming_llm.stream(prompt):
    print(chunk.content, end="", flush=True)
    time.sleep(0.01)  # Small delay to visualize streaming

print("\n" + "-" * 50)
print("✅ Streaming complete!")

### Performance Comparison

Let's compare Claude 4.5 Haiku with different temperature settings for various tasks."

In [ ]:
import time

def test_model_performance(prompt, temperatures=[0.1, 0.5, 0.9]):
    """Test the same prompt with different temperature settings"""
    results = {}

    for temp in temperatures:
        print(f"\n🌡️ Testing with temperature: {temp}")
        print("-" * 40)

        test_llm = ChatOpenAI(
            model="anthropic/claude-haiku-4.5",
            openai_api_key=api_key,
            openai_api_base="https://openrouter.ai/api/v1",
            temperature=temp,
            max_tokens=100
        )

        start_time = time.time()
        response = test_llm.invoke(prompt)
        end_time = time.time()

        results[temp] = {
            'response': response.content,
            'time': end_time - start_time
        }

        print(f"Response: {response.content}")
        print(f"⏱️ Time taken: {end_time - start_time:.2f} seconds")

    return results

# Test with a creative prompt
creative_prompt = "Write a haiku about artificial intelligence."
print("🎨 CREATIVE TASK: Writing a haiku about AI")
print("=" * 50)

creative_results = test_model_performance(creative_prompt)

print("\n\n📊 ANALYTICAL TASK: Explaining a concept")
print("=" * 50)

# Test with an analytical prompt
analytical_prompt = "Explain the concept of recursion in programming."
analytical_results = test_model_performance(analytical_prompt)

## Summary

This notebook demonstrated the capabilities of Claude 4.5 Haiku through OpenRouter, including:

1. **Basic Setup**: How to configure and initialize the model
2. **Simple Chat**: Basic conversational interactions
3. **Multilingual Support**: Testing with Hindi and Spanish
4. **Agent Workflows**: Creating agents with tool calling capabilities
5. **Tavily Integration**: Real-time search functionality
6. **Custom Tools**: Building and integrating custom tools
7. **Parameter Tuning**: Controlling creativity vs. factual accuracy
8. **Code Generation**: Programming task assistance
9. **Few-Shot Learning**: Pattern-based responses
10. **Structured Output**: JSON generation
11. **Streaming**: Real-time response generation
12. **Performance Testing**: Comparing different configurations

Claude 4.5 Haiku offers excellent performance for a wide range of tasks while being cost-effective and fast. It's particularly well-suited for applications requiring quick responses and efficient processing."